<a href="https://colab.research.google.com/github/disha2sinha/Movie-Recommendation-System/blob/master/Content_Based_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from ast import literal_eval
import pandas as pd
import math
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1P24X15eU7o234y_66hUYV9IDT5BqMVnG'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('PopularMovies.csv') 
movies_data = pd.read_csv('PopularMovies.csv',engine='python',index_col=0)
movies_data.head()

,movieId,title,budget,profit,score,release_year,release_day,genres,keywords,production_countries,production_companies,original_language,runtime,content,status,cast,director
0,135887,Minions,74000000,1.082731e+09,0.631865,2015,Wednesday,Family|Animation|Adventure|Comedy,"['assistant', 'aftercreditsstinger', 'duringcr...",United States of America,Universal Pictures|Illumination Entertainment,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Released,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda
1,143355,Wonder Woman,149000000,6.715804e+08,0.595920,2017,Tuesday,Action|Adventure|Fantasy,"['dc comics', 'hero', 'greek mythology', 'isla...",United States of America,Dune Entertainment|Atlas Entertainment|Warner ...,English,141.0,An Amazon princess comes to the world of Man t...,Released,Gal Gadot|Chris Pine|Robin Wright|Danny Huston...,Patty Jenkins
2,296,Pulp Fiction,8000000,2.059288e+08,0.590885,1994,Saturday,Thriller|Crime,"['transporter', 'brothel', 'drug dealer', 'box...",United States of America,Miramax Films|A Band Apart|Jersey Films,English,154.0,"A burger-loving hit man, his philosophical par...",Released,John Travolta|Samuel L. Jackson|Uma Thurman|Br...,Quentin Tarantino
3,58559,The Dark Knight,185000000,8.195584e+08,0.577222,2008,Wednesday,Drama|Action|Crime|Thriller,"['dc comics', 'crime fighter', 'secret identit...",United Kingdom|United States of America,DC Comics|Legendary Pictures|Warner Bros.|DC E...,English,152.0,Batman raises the stakes in his war on crime. ...,Released,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan
4,318,The Shawshank Redemption,25000000,3.341469e+06,0.568631,1994,Friday,Drama|Crime,"['prison', 'corruption', 'police brutality', '...",United States of America,Castle Rock Entertainment|Warner Bros.,English,142.0,Framed in the 1940s for the double murder of h...,Released,Tim Robbins|Morgan Freeman|Bob Gunton|Clancy B...,Frank Darabont


In [4]:
movies_data.shape

(46904, 17)

In [5]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46904 entries, 0 to 46903
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movieId               46904 non-null  int64  
 1   title                 46904 non-null  object 
 2   budget                46904 non-null  int64  
 3   profit                46904 non-null  float64
 4   score                 46904 non-null  float64
 5   release_year          46904 non-null  object 
 6   release_day           46904 non-null  object 
 7   genres                44380 non-null  object 
 8   keywords              46904 non-null  object 
 9   production_countries  40388 non-null  object 
 10  production_companies  34566 non-null  object 
 11  original_language     46774 non-null  object 
 12  runtime               46904 non-null  float64
 13  content               45918 non-null  object 
 14  status                46904 non-null  object 
 15  cast               

In [6]:
movies_data=movies_data.replace(np.nan,'Unknown',regex=True)
movies_data['release_year']=movies_data['release_year'].replace('NaT','1874',regex=True)

In [7]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46904 entries, 0 to 46903
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movieId               46904 non-null  int64  
 1   title                 46904 non-null  object 
 2   budget                46904 non-null  int64  
 3   profit                46904 non-null  float64
 4   score                 46904 non-null  float64
 5   release_year          46904 non-null  object 
 6   release_day           46904 non-null  object 
 7   genres                46904 non-null  object 
 8   keywords              46904 non-null  object 
 9   production_countries  46904 non-null  object 
 10  production_companies  46904 non-null  object 
 11  original_language     46904 non-null  object 
 12  runtime               46904 non-null  float64
 13  content               46904 non-null  object 
 14  status                46904 non-null  object 
 15  cast               

In [8]:
movies_data['production_companies']=movies_data['production_companies'].str.replace(" ","")
movies_data['production_companies']=movies_data['production_companies'].str.replace("|","  ")
movies_data['production_countries']=movies_data['production_countries'].str.replace(" ","")
movies_data['production_countries']=movies_data['production_countries'].str.replace("|","  ")
movies_data['cast']=movies_data['cast'].str.replace(" ","")
movies_data['cast']=movies_data['cast'].str.replace("|","  ")
movies_data['genres']=movies_data['genres'].str.replace("|"," ")
movies_data['director']=movies_data['director'].str.replace(" ","")
movies_data['keywords']=movies_data['keywords'].apply(lambda x:x[1:-1])
movies_data['keywords']=movies_data['keywords'].str.replace(',',"  ")
movies_data['keywords']=movies_data['keywords'].str.replace("'","")
movies_data.head(10)

,movieId,title,budget,profit,score,release_year,release_day,genres,keywords,production_countries,production_companies,original_language,runtime,content,status,cast,director
0,135887,Minions,74000000,1.082731e+09,0.631865,2015,Wednesday,Family Animation Adventure Comedy,assistant aftercreditsstinger duringcredit...,UnitedStatesofAmerica,UniversalPictures IlluminationEntertainment,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Released,SandraBullock JonHamm MichaelKeaton Allison...,KyleBalda
1,143355,Wonder Woman,149000000,6.715804e+08,0.595920,2017,Tuesday,Action Adventure Fantasy,dc comics hero greek mythology island ...,UnitedStatesofAmerica,DuneEntertainment AtlasEntertainment WarnerB...,English,141.0,An Amazon princess comes to the world of Man t...,Released,GalGadot ChrisPine RobinWright DannyHuston ...,PattyJenkins
2,296,Pulp Fiction,8000000,2.059288e+08,0.590885,1994,Saturday,Thriller Crime,transporter brothel drug dealer boxer ...,UnitedStatesofAmerica,MiramaxFilms ABandApart JerseyFilms,English,154.0,"A burger-loving hit man, his philosophical par...",Released,JohnTravolta SamuelL.Jackson UmaThurman Bru...,QuentinTarantino
3,58559,The Dark Knight,185000000,8.195584e+08,0.577222,2008,Wednesday,Drama Action Crime Thriller,dc comics crime fighter secret identity ...,UnitedKingdom UnitedStatesofAmerica,DCComics LegendaryPictures WarnerBros. DCEn...,English,152.0,Batman raises the stakes in his war on crime. ...,Released,ChristianBale MichaelCaine HeathLedger Aaro...,ChristopherNolan
4,318,The Shawshank Redemption,25000000,3.341469e+06,0.568631,1994,Friday,Drama Crime,prison corruption police brutality priso...,UnitedStatesofAmerica,CastleRockEntertainment WarnerBros.,English,142.0,Framed in the 1940s for the double murder of h...,Released,TimRobbins MorganFreeman BobGunton ClancyBr...,FrankDarabont
5,115617,Big Hero 6,165000000,4.871054e+08,0.556406,2014,Friday,Adventure Family Animation Action Comedy,brother brother relationship hero talent ...,UnitedStatesofAmerica,WaltDisneyPictures WaltDisneyAnimationStudios,English,102.0,The special bond that develops between plus-si...,Released,ScottAdsit RyanPotter DanielHenney T.J.Mill...,ChrisWilliams
6,122904,Deadpool,58000000,7.251130e+08,0.545813,2016,Tuesday,Action Adventure Comedy,anti hero mercenary marvel comic superhe...,UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation MarvelEnte...,English,108.0,Deadpool tells the origin story of former Spec...,Released,RyanReynolds MorenaBaccarin EdSkrein T.J.Mi...,TimMiller
7,122918,Guardians of the Galaxy Vol. 2,200000000,6.634161e+08,0.544638,2017,Wednesday,Action Adventure Comedy Science Fiction,sequel superhero based on comic misfit ...,UnitedStatesofAmerica,WaltDisneyPictures MarvelStudios,English,137.0,The Guardians must fight to keep their newfoun...,Released,ChrisPratt ZoeSaldana DaveBautista VinDiese...,JamesGunn
8,168366,Beauty and the Beast,160000000,1.102886e+09,0.536204,2017,Thursday,Family Fantasy Romance,france magic castle fairy tale musical...,UnitedKingdom UnitedStatesofAmerica,WaltDisneyPictures MandevilleFilms,English,129.0,A live-action adaptation of Disney's version o...,Released,EmmaWatson DanStevens LukeEvans KevinKline ...,BillCondon
9,112556,Gone Girl,61000000,3.083304e+08,0.535379,2014,Wednesday,Mystery Thriller Drama,based on novel marriage crisis disappearan...,UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation RegencyEnt...,English,145.0,With his wife's disappearance having become th...,Released,BenAffleck RosamundPike CarrieCoon NeilPatr...,DavidFincher


In [9]:
movies_data['keywords']=movies_data['keywords']+" " +movies_data['cast']+" "+movies_data['production_companies']+" "+movies_data['production_countries']+" "+movies_data['genres']+" "+movies_data['original_language']+" "+movies_data['release_day']+" "+movies_data['content']+" "+movies_data['director']
movies_data['keywords']=movies_data['keywords'].replace({"[^A-Za-z0-9]+": " "},regex=True)
movies_data['keywords'][0]

'assistant aftercreditsstinger duringcreditsstinger evil mastermind minions 3d SandraBullock JonHamm MichaelKeaton AllisonJanney SteveCoogan JenniferSaunders GeoffreyRush SteveCarell PierreCoffin KatyMixon MichaelBeattie HiroyukiSanada DaveRosenbaum AlexDowding PaulThornley KyleBalda AvaAcres UniversalPictures IlluminationEntertainment UnitedStatesofAmerica Family Animation Adventure Comedy English Wednesday Minions Stuart Kevin and Bob are recruited by Scarlet Overkill a super villain who alongside her inventor husband Herb hatches a plot to take over the world Before Gru they had a history of bad bosses KyleBalda'

In [10]:
vectorizer=TfidfVectorizer(stop_words='english')
X=vectorizer.fit_transform(movies_data['keywords'])
#matrix=scipy.sparse.eye(46904)
tfidf_df=pd.DataFrame.sparse.from_spmatrix(X)

In [11]:
movies_df=movies_data[['movieId','budget','profit','score','release_year','runtime']]
movies_df=movies_df.join(tfidf_df)
movies_df.set_index('movieId',inplace=True)
movies_df.shape

(46904, 314604)

In [12]:
#movies_df.head()

In [ ]:
sse={}
for k in range(1,40):
  kmeans=KMeans(n_clusters=k,init='k-means++',max_iter=100).fit(movies_df)
  movies_df['clusters']=kmeans.labels_
  sse[k]=kmeans.inertia_
plt.plot(list(sse.keys()))